1.Install Dependencies 

In [2]:
%pip install tensorflow==2.15.0
%pip install gym
%pip install  gym[atari]
%pip install keras-r12

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0)
ERROR: No matching distribution found for tensorflow==2.5.0

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement keras-r12 (from versions: none)
ERROR: No matching distribution found for keras-r12

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


2.Test Random Environment with OpenAI

In [1]:
#build environment in OpenAI
import gym
#Takes random steps to see how random model performs
import random

In [2]:
#Generates Atari environment only supports 'human' mode
env = gym.make('Assault-v4', render_mode='human')
#Grab items out of environment. Image shape
height, width, channels = env.observation_space.shape
#Different type of movements agent can take 
actions = env.action_space.n

In [9]:
#shows how many actions and action names
actions
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'UP', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [3]:
episodes = 5

for episode in range(1, episodes+1):
    #reseting env at begining of episode 
    state = env.reset()
    done = False 
    score = 0

    while not done:
        #render env after resetting
        env.render()
        #taking random action to see how it impacts env
        action = random.choice([0,1,2,3,4,5,6])
        #applying action to env
        #values represent estimations of future returns associated with state-action pairs
        n_state, reward, done, truncated, info = env.step(action)
        score+=reward
    #printing final reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


c:\Users\lizet\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:289: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
c:\Users\lizet\AppData\Local\Programs\Python\Python311\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:231.0
Episode:2 Score:273.0
Episode:3 Score:273.0
Episode:4 Score:168.0
Episode:5 Score:315.0


In [7]:
import numpy as np
#Allows to bring sequrntial API's to build sequencials models 
from tensorflow.python.keras import Sequential 
#Scans images and sees how model performs
from tensorflow.python.keras.layers import Dense, Flatten, Convolution2D
#Used when model compiled
from tensorflow.python.keras.optimizer_v1 import Adam

In [10]:
def build_model(height, width, channels, actions):
    model = Sequential()
    #image based model: use convolution layers and then flatten them 
    #32 filter that are trained to identify where things are 
    #8 by 8 filters
    #strides: how big of steps 
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height,width,channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    #Fully connected layer: each unit in a particular layer is connected to every single unit in the next layer 
    #image to action
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [11]:
#instanciate model
model = build_model(height,width,channels,actions)

In [12]:
#model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 1

Build Agent with Keras-RL

In [ ]:
#Reinforcement agent 
from rl.agents import DQNAAgent
#Allow hold knowledge buffer. Hold info from previous games
from rl.memory import SequentialMemory
#Greedy allows find the best reward
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
